In [1]:
import sys
sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# all envs

from environment.deepqlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv
from utils.reader import get_yaml_path, read_file

In [4]:
server_address = "localhost:50060"
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name)
env.connect_to_client()

2025-11-11 18:47:06,393 — INFO — ✓ Connected to localhost:50060



In [5]:
config_path = get_yaml_path("src", "scripts", "resources", "generated", "dq-obstacle-avoidance-test", "environment_0001.yml")
config = read_file(config_path)
# print(config)

In [6]:
env.init(config)

(True, '')

In [7]:
from agent.scala_dqagent import DQAgent
from training.dqnetwork import DQNetwork

2025-11-11 18:47:06.592516: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 18:47:06.627078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-11 18:47:09.554803: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
neuron_count_per_hidden_layer = [64, 32]

In [9]:
agent1 = DQAgent(
    env,
    agent_id="00000000-0000-0000-0000-000000000001",
    action_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
    target_model=DQNetwork(
        env.observation_space.shape,
        neuron_count_per_hidden_layer,
        env.action_space.n,
        summary=False,
    ),
)
agents = [agent1]

2025-11-11 18:47:10.570905: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
path = get_yaml_path("src", "scripts", "resources", "generated", "checkpoints", "deep", "high_penalty", "oa_final")
for agent in agents:
    agent.load(path)

/nix/store/pwrhqv9fy9lbybjz9c1w283z234lzksr-python3-3.13.8-env/lib/python3.13/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
import time

from training.multi_agent_dqlearning import DQLearning

trainer = DQLearning(
    env,
    agents,
    configs = [],
)

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/nix/store/pwrhqv9fy9lbybjz9c1w283z234lzksr-python3-3.13.8-env/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [12]:
trainer.play_with_pygame(fps=60)

2025-11-11 18:47:42,566 — INFO — Episode 1/1 - Reward: -58.849931655731936
